# Group 21 - We're Kenough

- Angelica Ricci: 105181
- Mattia Milone: 105633
- Michele Lamon: 105251

Estimated time running:
+ intel 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz 16GB ram: 3 hours 

# Short notebook - 1

## Importing all the libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from datetime import datetime

import lightgbm as lgb
from lightgbm import LGBMRegressor
import catboost
from catboost import CatBoostRegressor
from catboost import Pool
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

import warnings
import os
import re
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Uploading the data

In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

In [3]:
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

In [4]:
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

In [5]:
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

In [6]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

In [7]:
test2 = pd.read_csv('test.csv')
test2['time'] = pd.to_datetime(test2['time'])

## Preparation of the data

In [8]:
train_a['time'] = pd.to_datetime(train_a['time'])
train_b['time'] = pd.to_datetime(train_b['time'])
train_c['time'] = pd.to_datetime(train_c['time'])

X_train_estimated_a['date_forecast'] = pd.to_datetime(X_train_estimated_a['date_forecast'])
X_train_estimated_b['date_forecast'] = pd.to_datetime(X_train_estimated_b['date_forecast'])
X_train_estimated_c['date_forecast'] = pd.to_datetime(X_train_estimated_c['date_forecast'])

X_train_observed_a['date_forecast'] = pd.to_datetime(X_train_observed_a['date_forecast'])
X_train_observed_b['date_forecast'] = pd.to_datetime(X_train_observed_b['date_forecast'])
X_train_observed_c['date_forecast'] = pd.to_datetime(X_train_observed_c['date_forecast'])

X_test_estimated_a['date_forecast'] = pd.to_datetime(X_test_estimated_a['date_forecast'])
X_test_estimated_b['date_forecast'] = pd.to_datetime(X_test_estimated_b['date_forecast'])
X_test_estimated_c['date_forecast'] = pd.to_datetime(X_test_estimated_c['date_forecast'])

In [9]:
X_train_estimated_a['forecast_age'] = (X_train_estimated_a['date_forecast']-X_train_estimated_a['date_calc']).dt.seconds/3600
del X_train_estimated_a['date_calc']
X_test_estimated_a['forecast_age'] = (X_test_estimated_a['date_forecast']-X_test_estimated_a['date_calc']).dt.seconds/3600
del X_test_estimated_a['date_calc']

X_train_estimated_b['forecast_age'] = (X_train_estimated_b['date_forecast']-X_train_estimated_b['date_calc']).dt.seconds/3600
del X_train_estimated_b['date_calc']
X_test_estimated_b['forecast_age'] = (X_test_estimated_b['date_forecast']-X_test_estimated_b['date_calc']).dt.seconds/3600
del X_test_estimated_b['date_calc']

X_train_estimated_c['forecast_age'] = (X_train_estimated_c['date_forecast']-X_train_estimated_c['date_calc']).dt.seconds/3600
del X_train_estimated_c['date_calc']
X_test_estimated_c['forecast_age'] = (X_test_estimated_c['date_forecast']-X_test_estimated_c['date_calc']).dt.seconds/3600
del X_test_estimated_c['date_calc']

In [10]:
X_train_observed_a.set_index('date_forecast', inplace=True)
X_train_observed_b.set_index('date_forecast', inplace=True)
X_train_observed_c.set_index('date_forecast', inplace=True)

X_train_estimated_a.set_index('date_forecast', inplace=True)
X_train_estimated_b.set_index('date_forecast', inplace=True)
X_train_estimated_c.set_index('date_forecast', inplace=True)

X_test_estimated_a.set_index('date_forecast', inplace=True)
X_test_estimated_b.set_index('date_forecast', inplace=True)
X_test_estimated_c.set_index('date_forecast', inplace=True)

In [11]:
X_train_observed_a['sensor_A'] = 1
X_train_observed_a['sensor_B'] = 0
X_train_observed_a['sensor_C'] = 0
X_train_observed_a['month'] = X_train_observed_a.index.strftime('%m')
X_train_observed_a['hour'] = X_train_observed_a.index.strftime('%H')
X_train_observed_a['forecast_age'] = 0
X_train_observed_a['is_obv'] = 1

X_train_observed_b['sensor_A'] = 0
X_train_observed_b['sensor_B'] = 1
X_train_observed_b['sensor_C'] = 0
X_train_observed_b['month'] = X_train_observed_b.index.strftime('%m')
X_train_observed_b['hour'] = X_train_observed_b.index.strftime('%H')
X_train_observed_b['forecast_age'] = 0
X_train_observed_b['is_obv'] = 1

X_train_observed_c['sensor_A'] = 0
X_train_observed_c['sensor_B'] = 0
X_train_observed_c['sensor_C'] = 1
X_train_observed_c['month'] = X_train_observed_c.index.strftime('%m')
X_train_observed_c['hour'] = X_train_observed_c.index.strftime('%H')
X_train_observed_c['forecast_age'] = 0
X_train_observed_c['is_obv'] = 1

In [12]:
X_train_estimated_a['sensor_A'] = 1
X_train_estimated_a['sensor_B'] = 0
X_train_estimated_a['sensor_C'] = 0
X_train_estimated_a['month'] = X_train_estimated_a.index.strftime('%m')
X_train_estimated_a['hour'] = X_train_estimated_a.index.strftime('%H')
X_train_estimated_a['is_obv'] = 0

X_train_estimated_b['sensor_A'] = 0
X_train_estimated_b['sensor_B'] = 1
X_train_estimated_b['sensor_C'] = 0
X_train_estimated_b['month'] = X_train_estimated_b.index.strftime('%m')
X_train_estimated_b['hour'] = X_train_estimated_b.index.strftime('%H')
X_train_estimated_b['is_obv'] = 0

X_train_estimated_c['sensor_A'] = 0
X_train_estimated_c['sensor_B'] = 0
X_train_estimated_c['sensor_C'] = 1
X_train_estimated_c['month'] = X_train_estimated_c.index.strftime('%m')
X_train_estimated_c['hour'] = X_train_estimated_c.index.strftime('%H')
X_train_estimated_c['is_obv'] = 0

In [13]:
X_test_estimated_a['sensor_A'] = 1
X_test_estimated_a['sensor_B'] = 0
X_test_estimated_a['sensor_C'] = 0
X_test_estimated_a['month'] = X_test_estimated_a.index.strftime('%m')
X_test_estimated_a['hour'] = X_test_estimated_a.index.strftime('%H')
X_test_estimated_a['is_obv'] = 0

X_test_estimated_b['sensor_A'] = 0
X_test_estimated_b['sensor_B'] = 1
X_test_estimated_b['sensor_C'] = 0
X_test_estimated_b['month'] = X_test_estimated_b.index.strftime('%m')
X_test_estimated_b['hour'] = X_test_estimated_b.index.strftime('%H')
X_test_estimated_b['is_obv'] = 0

X_test_estimated_c['sensor_A'] = 0
X_test_estimated_c['sensor_B'] = 0
X_test_estimated_c['sensor_C'] = 1
X_test_estimated_c['month'] = X_test_estimated_c.index.strftime('%m')
X_test_estimated_c['hour'] = X_test_estimated_c.index.strftime('%H')
X_test_estimated_c['is_obv'] = 0

In [14]:
X_train_observed_a[["month", "hour"]] = X_train_observed_a[["month", "hour"]].apply(pd.to_numeric)
X_train_estimated_a[["month", "hour"]] = X_train_estimated_a[["month", "hour"]].apply(pd.to_numeric)
X_test_estimated_a[["month", "hour"]] = X_test_estimated_a[["month", "hour"]].apply(pd.to_numeric)

X_train_observed_b[["month", "hour"]] = X_train_observed_b[["month", "hour"]].apply(pd.to_numeric)
X_train_estimated_b[["month", "hour"]] = X_train_estimated_b[["month", "hour"]].apply(pd.to_numeric)
X_test_estimated_b[["month", "hour"]] = X_test_estimated_b[["month", "hour"]].apply(pd.to_numeric)

X_train_observed_c[["month", "hour"]] = X_train_observed_c[["month", "hour"]].apply(pd.to_numeric)
X_train_estimated_c[["month", "hour"]] = X_train_estimated_c[["month", "hour"]].apply(pd.to_numeric)
X_test_estimated_c[["month", "hour"]] = X_test_estimated_c[["month", "hour"]].apply(pd.to_numeric)

In [15]:
X_train_observed_a = X_train_observed_a.resample('H').median()
X_train_observed_b = X_train_observed_b.resample('H').median()
X_train_observed_c = X_train_observed_c.resample('H').median()
X_train_estimated_a = X_train_estimated_a.resample('H').median()
X_train_estimated_b = X_train_estimated_b.resample('H').median()
X_train_estimated_c = X_train_estimated_c.resample('H').median()
X_test_estimated_a = X_test_estimated_a.resample('H').median()
X_test_estimated_b = X_test_estimated_b.resample('H').median()
X_test_estimated_c = X_test_estimated_c.resample('H').median()

In [16]:
train_b_new = train_b.copy()

train_b_new['diff1'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(1)
train_b_new['diff3'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(3)
train_b_new['diff5'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(5)
train_b_new['diff7'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(7)
train_b_new['diff10'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(10)
train_b_new['diff15'] = train_b_new['pv_measurement'] - train_b_new['pv_measurement'].shift(15)

train_b_new['toDrop'] = ((train_b_new['diff1'] + train_b_new[
    'diff3'] + train_b_new['diff5'] + train_b_new['diff7'] + train_b_new['diff10'] + train_b_new['diff15']) == 0)

train_b_new = train_b_new[train_b_new['toDrop'] != True]
train_b_new = train_b_new.drop(columns=['diff1', 'diff3', 'diff5', 'diff7', 'diff10', 'diff15', 'toDrop'])

In [17]:
train_c_new = train_c.copy()

train_c_new['diff1'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(1)
train_c_new['diff3'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(3)
train_c_new['diff5'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(5)
train_c_new['diff7'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(7)
train_c_new['diff10'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(10)
train_c_new['diff15'] = train_c_new['pv_measurement'] - train_c_new['pv_measurement'].shift(15)

train_c_new['toDrop'] = ((train_c_new['diff1'] + train_c_new[
    'diff3'] + train_c_new['diff5'] + train_c_new['diff7'] + train_c_new['diff10'] + train_c_new['diff15']) == 0)

train_c_new = train_c_new[train_c_new['toDrop'] != True]
train_c_new = train_c_new.drop(columns=['diff1', 'diff3', 'diff5', 'diff7', 'diff10', 'diff15', 'toDrop'])

In [18]:
y_train_observed_a = train_a.merge(X_train_observed_a, left_on='time', right_on='date_forecast', how='right')
y_train_observed_b = train_b_new.merge(X_train_observed_b, left_on='time', right_on='date_forecast', how='right')
y_train_observed_c = train_c_new.merge(X_train_observed_c, left_on='time', right_on='date_forecast', how='right')

y_train_obv = pd.concat([y_train_observed_a,y_train_observed_b,y_train_observed_c])

In [19]:
y_train_estimated_a = train_a.merge(X_train_estimated_a, left_on='time', right_on='date_forecast', how='right')
y_train_estimated_b = train_b_new.merge(X_train_estimated_b, left_on='time', right_on='date_forecast', how='right')
y_train_estimated_c = train_c_new.merge(X_train_estimated_c, left_on='time', right_on='date_forecast', how='right')

y_train_est = pd.concat([y_train_estimated_a,y_train_estimated_b,y_train_estimated_c])

In [20]:
test = pd.concat([X_test_estimated_a,X_test_estimated_b,X_test_estimated_c])

## NaN Values

In [21]:
feature_with_nan_obv = [features for features in y_train_obv.columns if y_train_obv[features].isnull().sum()>1]
feature_with_nan_est =[features for features in y_train_est.columns if y_train_est[features].isnull().sum()>1]

In [22]:
del y_train_obv['snow_density:kgm3']
del y_train_est['snow_density:kgm3']
del test['snow_density:kgm3']

In [23]:
columns_to_delete=['ceiling_height_agl:m','snow_drift:idx','elevation:m','cloud_base_agl:m']
y_train_obv.drop(columns=columns_to_delete, inplace=True)
y_train_est.drop(columns=columns_to_delete, inplace=True)
test.drop(columns=columns_to_delete, inplace=True)

In [24]:
y_train_obv = y_train_obv.dropna()
y_train_est = y_train_est.dropna()

## Train and test sets

In [25]:
train=pd.concat([y_train_obv,y_train_est])
train=train.set_index('time')

In [26]:
col = test.pop('forecast_age')
test.insert(45, col.name, col)

In [27]:
train['day_of_week'] = train.index.dayofweek
test['day_of_week'] = test.index.dayofweek
train['day_of_year'] = train.index.dayofyear
test['day_of_year'] = test.index.dayofyear

In [28]:
season_mapping = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1,
}

# Create a new column 'Season' based on the mapping
train['season'] = train['month'].map(season_mapping)
test['season'] = test['month'].map(season_mapping)

In [29]:
train = pd.get_dummies(train, columns=['is_day:idx'])
test = pd.get_dummies(test, columns=['is_day:idx'])
train = pd.get_dummies(train, columns=['is_in_shadow:idx'])
test = pd.get_dummies(test, columns=['is_in_shadow:idx'])

In [30]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [31]:
X_train = train.loc[:, train.columns != "pv_measurement"]
y_train = train['pv_measurement']

In [32]:
# Create cyclical features for day of the week
X_train['day_of_week_sin'] = np.sin(2 * np.pi * X_train['day_of_week'] / 7)
X_train['day_of_week_cos'] = np.cos(2 * np.pi * X_train['day_of_week'] / 7)

# Create cyclical features for month
X_train['month_sin'] = np.sin(2 * np.pi * X_train['month'] / 12)
X_train['month_cos'] = np.cos(2 * np.pi * X_train['month'] / 12)

# Create cyclical features for day of the week
test['day_of_week_sin'] = np.sin(2 * np.pi * test['day_of_week'] / 7)
test['day_of_week_cos'] = np.cos(2 * np.pi * test['day_of_week'] / 7)

# Create cyclical features for month
test['month_sin'] = np.sin(2 * np.pi * test['month'] / 12)
test['month_cos'] = np.cos(2 * np.pi * test['month'] / 12)

In [33]:
y_train_transformed = np.sqrt(y_train)

## Preparation of the test set and the submission file base

In [34]:
X_test = test2.merge(test, left_on='time', right_on='date_forecast', how='left')
X_test = pd.get_dummies(X_test, columns=['location'], prefix='sensor2')

In [35]:
X_test = X_test[(X_test['sensor_A'] == X_test['sensor2_A']) &
                 (X_test['sensor_B'] == X_test['sensor2_B']) &
                 (X_test['sensor_C'] == X_test['sensor2_C'])]

In [36]:
del X_test['sensor2_A']
del X_test['sensor2_B']
del X_test['sensor2_C']

In [37]:
X_test = X_test.sort_values(by='id')

In [38]:
del X_test['id']
del X_test['prediction']

In [39]:
X_test=X_test.set_index('time')

In [40]:
test = test.rename_axis('time')
X_test = X_test.rename_axis('time')

In [41]:
X_train.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in X_train.columns]
X_test.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in X_test.columns]
test.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', col) for col in test.columns]

## Generic Functions

In [42]:
def pred_to_df(array_pred, squared=False):
    if squared==True:
        array_pred = array_pred**2

    df_predicitons = pd.DataFrame(array_pred)
    df_predicitons = df_predicitons.applymap(lambda x: max(0, x))
    df_predicitons.iloc[:,0] = df_predicitons.iloc[:,0].apply(lambda x: 0 if 0 <= x <= 1 else x)
    df_predicitons.rename(columns={df_predicitons.columns[0]: 'values'}, inplace=True)
    df_predicitons = df_predicitons.rename_axis('id')
    return df_predicitons

In [43]:
def plot_pred(pred_list, name_list, a1=0, a2=2160):
    """
    Plots the given prediction dataframes from a1 to a2 with the given labels

    Parameters:
    - pred_list (list of df): list of prediction dataframes.
    - name_list (list of str): list of labels (same order as pred_list).
    - a1 (int, optional): beginning of viewing window (0).
    - a2 (int, optional): end of viewing window (2160).

    Returns:
    - nothing

    Example:
    
    plot_pred([best_of_best, pazzo, xgb_sum_pred], ['best_of_best', 'pazzo', 'xgb_sum_pred'], 100, 300)
    
    """

    new_dataframe = pd.DataFrame()

    for i in range(len(pred_list)):
        new_dataframe[name_list[i]] = pred_list[i]['values']

    plt.figure(figsize=(20,9))

    for i in range(len(pred_list)):
        new_dataframe[name_list[i]][a1:a2].plot(label=name_list[i])

    plt.legend()
    plt.show()

## Preparation of the models

Hyperparameters for the models used.

In [44]:
dict_model_param = {
    'lgbm': {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.01,
        'feature_fraction': 0.7,
        'feature_pre_filter': True,
        'bagging_freq': 5,
        'bagging_fraction': 0.75
    },
    'lgbm_reg': {
        'learning_rate': 0.07019571273972466,
        'n_estimators': 350,
        'max_depth': 10,
        'min_child_samples': 29,
        'feature_fraction': 0.8244973703390805,
        'bagging_fraction': 0.753170749901516,
        'reg_alpha': 0.9719949744352129,
        'reg_lambda': 0.8504246860234232,
        'min_split_gain': 0.36086476058243666,
        'max_bin': 62,
        'min_child_weight': 0.256068322761324,
        'subsample_for_bin': 30
    },
    'catboost': { #MIGLIORI
        'iterations': 1000, 
        'depth': 6, 
        'learning_rate': 0.1, 
        'loss_function': 'MAE'
    },
    'catboost_opt': {
        'depth': 6,
        'iterations': 4598,
        'learning_rate': 0.11381056696717655
    },
    'xgboost' : {
        'colsample_bytree': 0.8,
        'learning_rate': 0.01,
        'max_depth': 5,
        'n_estimators': 1000,
        'subsample': 0.8
    },
    'xgboost2' : { #MIGLIORI
        'colsample_bytree': 0.70468596253168,
        'learning_rate': 0.00931602383319447,
        'max_depth': 7,
        'n_estimators': 56,
        'subsample': 0.9610097364456339
    },
    'xgboost3':{
        'colsample_bytree': 0.7224776657906091,
        'learning_rate': 0.044779531399174476,
        'max_depth': 10,
        'n_estimators': 906,
        'subsample': 0.706461320142151
    },
    'randomforest_opt' :{
        'max_depth': 26,
        'max_features': 0.4942262677968311,
        'min_samples_leaf': 6,
        'min_samples_split': 8,
        'n_estimators': 1809
    }
}

## Models Predictions

In [45]:
def pred_lgbm(X_train, y_train, test=None, params=None, weights=None):
    # Create a LightGBM dataset for the training set
    train_data = lgb.Dataset(X_train, label=y_train)

    # Define LightGBM parameters
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.01,
        'feature_fraction': 0.7,
        'feature_pre_filter':True,
        'bagging_freq': 5,
        'bagging_fraction': 0.75
    }

    # Train the LightGBM model
    num_round = 1000
    model = lgb.train(params, train_data, num_round)   

    if test is not None:
        # Make predictions on the test set
        predictions = model.predict(test, num_iteration=model.best_iteration)

        # Make predictions on the test set
        return model, model.predict(test, num_iteration=model.best_iteration)
    
    return model

In [46]:
def pred_lgbm_reg(X_train, y_train, test=None, params=0, weights=None):
    if params is None:
        params = dict_model_param['lgbm_reg']

    # Split your data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    model = LGBMRegressor(**params)

    # Specify the validation set using eval_set
    eval_set = [(X_val, y_val)]

    model.fit(X_train, y_train, eval_set=eval_set)

    if test is not None:
        # Make predictions on the test set
        predictions = model.predict(test)

        return model, predictions

    return model

In [47]:
def pred_catboost(X_train, y_train, test=None, params=0):

    if params ==0:
        params = dict_model_param['catboost']

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    train_dataset = Pool(data=X_train, label=y_train)
    test_dataset = Pool(data=X_test, label=y_test)

    model = CatBoostRegressor(**params)
    model.fit(train_dataset, eval_set=test_dataset, verbose=100)

    if test is not None:
        test_pool = Pool(data=test)
        predictions = model.predict(test_pool)

        return model, predictions

    return model

In [48]:
def pred_xgboost(X_train, y_train, test=None, params=0):
    
    if params ==0:
        params = dict_model_param['xgboost']
    
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42) 
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    
    if test is not None:
        predictions = model.predict(X_test)

        return model, predictions
    
    return model

In [49]:
def pred_randomforest(X_train, y_train, test=None, params=0):
    
    if params ==0:
        params = dict_model_param['randomforest']
    
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    
    if test is not None:
        predictions = model.predict(X_test)

        return model, predictions
    
    return model

In [50]:
def prediction_models(X_train, y_train, test=None, model='lgbm', params=None, weights=None):#, cat_features=0):
    if weights is None:
        weights = [1]*len(X_train)
    if model=='lgbm':
        return pred_lgbm(X_train, y_train, test, params, weights)
    if model=='catboost':
        return pred_catboost(X_train, y_train, test, params)
    if model=='xgboost':
        return pred_xgboost(X_train, y_train, test, params)
    if model=='randomforest':
        return pred_randomforest(X_train, y_train, test, params)
    if model=='lgbm_reg':
        return pred_lgbm_reg(X_train, y_train, test, params)

## Y_slided

In [51]:
def y_slided_prediction(X_train, y_train, test=test, model='lgbm', params=0, squared=False):
    if params==0:
        params = dict_model_param[model]

    Xy_train = X_train.copy()
    Xy_train['pv_measurement'] = y_train

    Xy_train_A = Xy_train[Xy_train['sensor_A'] == 1]
    Xy_train_B = Xy_train[Xy_train['sensor_B'] == 1]
    Xy_train_C = Xy_train[Xy_train['sensor_C'] == 1]

    Xy_train_A['previous_pv_measurement'] = Xy_train_A['pv_measurement'].shift(1, freq='H')
    Xy_train_B['previous_pv_measurement'] = Xy_train_B['pv_measurement'].shift(1, freq='H')
    Xy_train_C['previous_pv_measurement'] = Xy_train_C['pv_measurement'].shift(1, freq='H')

    X_train_y_slided = pd.concat([Xy_train_A, Xy_train_B, Xy_train_C])
    X_train_y_slided.dropna()
    y_train_y_slided = X_train_y_slided['pv_measurement']
    X_train_y_slided.drop(columns=['pv_measurement'], inplace=True)

    y_train_transformed = np.sqrt(y_train_y_slided) if squared else y_train_y_slided

    _,y_pred = prediction_models(X_train, y_train_transformed, test=test, model=model) #predizione non slided
    
    pred = pred_to_df(y_pred, squared)
    
    model2 = prediction_models(X_train_y_slided, y_train_y_slided, model=model)
    
    test_slide_A = test[test['sensor_A'] == 1]
    test_slide_B = test[test['sensor_B'] == 1]
    test_slide_C = test[test['sensor_C'] == 1]
    
    range2 = len(pred)
    range3 = range2 * 2
    
    pred_A = pred[:len(test_slide_A)-1]
    pred_B = pred[len(test_slide_A):len(test_slide_B)+len(test_slide_A)-1]
    pred_C = pred[len(test_slide_B)+len(test_slide_A):]
    
    '''
    pred_A = pred[0:1535]
    pred_B = pred[1536:3071]
    pred_C = pred[3072:]
    '''
    
    datasets = [test_slide_A, test_slide_B, test_slide_C]
    preds = [pred_A, pred_B, pred_C]

    result_all = []

    for i in [0, 1, 2]:

        dataset = datasets[i]
        pred_set = preds[i]

        result = []
        previous_pv_measurement = pred.values[0][0]

        result.append(previous_pv_measurement)
        max_time_diff = pd.Timedelta(hours=1)

        test_copy = dataset.copy()
        test_copy['previous_pv_measurement'] = 0

        for index in range(1, len(dataset)):
            if test_copy.index[index] - test_copy.index[index-1] > max_time_diff:
                previous_pv_measurement = pred_set.values[index][0]
            else:
                test_copy.loc[test_copy.index[index], 'previous_pv_measurement'] = previous_pv_measurement
                if model=='lgbm':
                    previous_pv_measurement = model2.predict(test_copy.iloc[index], num_iteration=model2.best_iteration)[0]
                else:
                    previous_pv_measurement = model2.predict(np.array(test_copy.iloc[index]).reshape(1, 59))[0]
                    
            result.append(previous_pv_measurement)

        result_all.append(result)
        
    flattened_list = [item for sublist in result_all for item in sublist]
    return pred_to_df(pd.DataFrame(flattened_list), False)

In [52]:
result_lgbm_slided = y_slided_prediction(X_train, y_train_transformed, test=test, model='lgbm', params=dict_model_param['lgbm'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7758
[LightGBM] [Info] Number of data points in the train set: 79218, number of used features: 58
[LightGBM] [Info] Start training from score 9.924039
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8013
[LightGBM] [Info] Number of data points in the train set: 79218, number of used features: 59
[LightGBM] [Info] Start training from score 9.924039


In [53]:
result_lgbm_slided = pred_to_df(result_lgbm_slided, squared= True)

In [54]:
result_lgbm_r_slided = y_slided_prediction(X_train, y_train, test=test, model='lgbm_reg', squared=False)

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-ch

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.82449

In [56]:
lgbm,result_lgbm = prediction_models(X_train, y_train_transformed, test=X_test, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7758
[LightGBM] [Info] Number of data points in the train set: 79218, number of used features: 58
[LightGBM] [Info] Start training from score 9.924039


In [57]:
result_lgbm = pred_to_df(result_lgbm, squared=True)

## Stacking

In [58]:
def stacking_models(base_models, X_train, y_train, test=None, final_regressor='linear', final_params={}):
    if final_regressor=='linear':
        final_model = LinearRegression()
    elif final_regressor == 'SVR':
        final_model = SVR(**final_params)
        
    stacking_regressor = StackingRegressor(
        estimators=base_models,
        final_estimator=final_model)
    
    stacking_regressor.fit(X_train, y_train)

    if X_test is not None:
        y_pred = stacking_regressor.predict(X_test)
        return stacking_regressor, y_pred
    else:
        return stacking_regressor

### Stacking 1

In [60]:
lgbm, lgb_pred = prediction_models(X_train, y_train_transformed, test=X_test, model='lgbm_reg', params=None)
lgb_pred = pred_to_df(lgb_pred, squared=True)

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-ch

In [61]:
catboostm, catboost_pred = prediction_models(X_train, y_train_transformed, test=X_test, model='catboost', params=dict_model_param['catboost'])
catboost_pred = pred_to_df(catboost_pred, squared=True)

0:	learn: 9.0652210	test: 9.0565245	best: 9.0565245 (0)	total: 155ms	remaining: 2m 35s
100:	learn: 2.0733148	test: 2.0567362	best: 2.0567362 (100)	total: 1.47s	remaining: 13.1s
200:	learn: 1.8829095	test: 1.8841427	best: 1.8841427 (200)	total: 2.71s	remaining: 10.8s
300:	learn: 1.8074775	test: 1.8289070	best: 1.8289070 (300)	total: 4.04s	remaining: 9.38s
400:	learn: 1.7689026	test: 1.8052966	best: 1.8052966 (400)	total: 5.26s	remaining: 7.86s
500:	learn: 1.7531543	test: 1.7968189	best: 1.7968066 (498)	total: 6.38s	remaining: 6.35s
600:	learn: 1.7364629	test: 1.7864132	best: 1.7864132 (597)	total: 7.49s	remaining: 4.97s
700:	learn: 1.7227136	test: 1.7781776	best: 1.7781776 (700)	total: 8.76s	remaining: 3.73s
800:	learn: 1.7090281	test: 1.7699329	best: 1.7699329 (800)	total: 9.96s	remaining: 2.47s
900:	learn: 1.6921185	test: 1.7613328	best: 1.7613328 (900)	total: 11.3s	remaining: 1.25s
999:	learn: 1.6786105	test: 1.7549962	best: 1.7549690 (995)	total: 12.7s	remaining: 0us

bestTest = 1.7

In [62]:
xgboost, xgboost_pred = prediction_models(X_train, y_train_transformed, test=X_test, model='xgboost', params=dict_model_param['xgboost2'])
xgboost_pred = pred_to_df(xgboost_pred, squared=True)

In [63]:
randomforest, randomforest_pred = prediction_models(X_train, y_train_transformed, test=X_test, model='randomforest', params=dict_model_param['randomforest_opt'])
randomforest_pred = pred_to_df(randomforest_pred, squared=True)

In [64]:
base_models = [
    ('lgbm', lgbm),
    ('catboost', catboostm),
    ('randomforest', randomforest)
]

In [65]:
final_model_1, final_pred_1 = stacking_models(base_models,
                                          X_train, y_train_transformed,
                                          test=X_test, final_regressor='linear',
                                          final_params={})

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-ch

136:	learn: 1.9917453	total: 2.17s	remaining: 13.6s
137:	learn: 1.9899005	total: 2.18s	remaining: 13.6s
138:	learn: 1.9891141	total: 2.2s	remaining: 13.6s
139:	learn: 1.9872972	total: 2.21s	remaining: 13.6s
140:	learn: 1.9856725	total: 2.23s	remaining: 13.6s
141:	learn: 1.9829191	total: 2.24s	remaining: 13.5s
142:	learn: 1.9779575	total: 2.25s	remaining: 13.5s
143:	learn: 1.9751523	total: 2.27s	remaining: 13.5s
144:	learn: 1.9706630	total: 2.28s	remaining: 13.5s
145:	learn: 1.9671126	total: 2.29s	remaining: 13.4s
146:	learn: 1.9652524	total: 2.31s	remaining: 13.4s
147:	learn: 1.9619327	total: 2.32s	remaining: 13.4s
148:	learn: 1.9601310	total: 2.33s	remaining: 13.3s
149:	learn: 1.9589545	total: 2.35s	remaining: 13.3s
150:	learn: 1.9551591	total: 2.36s	remaining: 13.3s
151:	learn: 1.9506114	total: 2.38s	remaining: 13.3s
152:	learn: 1.9483762	total: 2.39s	remaining: 13.2s
153:	learn: 1.9446391	total: 2.41s	remaining: 13.2s
154:	learn: 1.9417674	total: 2.42s	remaining: 13.2s
155:	learn: 1

303:	learn: 1.8149142	total: 4.54s	remaining: 10.4s
304:	learn: 1.8142967	total: 4.55s	remaining: 10.4s
305:	learn: 1.8142815	total: 4.57s	remaining: 10.4s
306:	learn: 1.8136887	total: 4.58s	remaining: 10.3s
307:	learn: 1.8128219	total: 4.59s	remaining: 10.3s
308:	learn: 1.8113340	total: 4.61s	remaining: 10.3s
309:	learn: 1.8107824	total: 4.62s	remaining: 10.3s
310:	learn: 1.8100839	total: 4.63s	remaining: 10.3s
311:	learn: 1.8093794	total: 4.65s	remaining: 10.3s
312:	learn: 1.8084826	total: 4.66s	remaining: 10.2s
313:	learn: 1.8082373	total: 4.67s	remaining: 10.2s
314:	learn: 1.8077990	total: 4.69s	remaining: 10.2s
315:	learn: 1.8077766	total: 4.7s	remaining: 10.2s
316:	learn: 1.8069710	total: 4.72s	remaining: 10.2s
317:	learn: 1.8059846	total: 4.73s	remaining: 10.1s
318:	learn: 1.8049258	total: 4.75s	remaining: 10.1s
319:	learn: 1.8043324	total: 4.76s	remaining: 10.1s
320:	learn: 1.8034837	total: 4.78s	remaining: 10.1s
321:	learn: 1.8029842	total: 4.79s	remaining: 10.1s
322:	learn: 1

468:	learn: 1.7564034	total: 6.86s	remaining: 7.76s
469:	learn: 1.7560449	total: 6.87s	remaining: 7.75s
470:	learn: 1.7560442	total: 6.89s	remaining: 7.74s
471:	learn: 1.7560136	total: 6.9s	remaining: 7.72s
472:	learn: 1.7557695	total: 6.92s	remaining: 7.7s
473:	learn: 1.7553185	total: 6.93s	remaining: 7.69s
474:	learn: 1.7551719	total: 6.94s	remaining: 7.67s
475:	learn: 1.7549059	total: 6.96s	remaining: 7.66s
476:	learn: 1.7544021	total: 6.97s	remaining: 7.64s
477:	learn: 1.7543940	total: 6.99s	remaining: 7.63s
478:	learn: 1.7543767	total: 7s	remaining: 7.61s
479:	learn: 1.7543757	total: 7.01s	remaining: 7.6s
480:	learn: 1.7543752	total: 7.03s	remaining: 7.58s
481:	learn: 1.7543742	total: 7.04s	remaining: 7.57s
482:	learn: 1.7543295	total: 7.06s	remaining: 7.55s
483:	learn: 1.7543187	total: 7.07s	remaining: 7.54s
484:	learn: 1.7542598	total: 7.08s	remaining: 7.52s
485:	learn: 1.7533241	total: 7.1s	remaining: 7.51s
486:	learn: 1.7532856	total: 7.11s	remaining: 7.49s
487:	learn: 1.75327

633:	learn: 1.7395058	total: 9.19s	remaining: 5.3s
634:	learn: 1.7394743	total: 9.2s	remaining: 5.29s
635:	learn: 1.7394593	total: 9.21s	remaining: 5.27s
636:	learn: 1.7393909	total: 9.23s	remaining: 5.26s
637:	learn: 1.7390701	total: 9.24s	remaining: 5.24s
638:	learn: 1.7381592	total: 9.26s	remaining: 5.23s
639:	learn: 1.7373696	total: 9.27s	remaining: 5.21s
640:	learn: 1.7373310	total: 9.28s	remaining: 5.2s
641:	learn: 1.7373293	total: 9.3s	remaining: 5.18s
642:	learn: 1.7369252	total: 9.31s	remaining: 5.17s
643:	learn: 1.7360217	total: 9.32s	remaining: 5.16s
644:	learn: 1.7356947	total: 9.34s	remaining: 5.14s
645:	learn: 1.7356944	total: 9.35s	remaining: 5.13s
646:	learn: 1.7356934	total: 9.37s	remaining: 5.11s
647:	learn: 1.7356769	total: 9.38s	remaining: 5.09s
648:	learn: 1.7353705	total: 9.39s	remaining: 5.08s
649:	learn: 1.7353702	total: 9.41s	remaining: 5.07s
650:	learn: 1.7353602	total: 9.42s	remaining: 5.05s
651:	learn: 1.7353596	total: 9.44s	remaining: 5.04s
652:	learn: 1.73

...

284:	learn: 1.9512799	total: 3.85s	remaining: 9.67s
285:	learn: 1.9507086	total: 3.87s	remaining: 9.65s
286:	learn: 1.9499504	total: 3.88s	remaining: 9.64s
287:	learn: 1.9494783	total: 3.89s	remaining: 9.63s
288:	learn: 1.9486336	total: 3.91s	remaining: 9.61s
289:	learn: 1.9482280	total: 3.92s	remaining: 9.6s
290:	learn: 1.9481587	total: 3.93s	remaining: 9.58s
291:	learn: 1.9481414	total: 3.94s	remaining: 9.56s
292:	learn: 1.9480544	total: 3.96s	remaining: 9.55s
293:	learn: 1.9476624	total: 3.97s	remaining: 9.54s
294:	learn: 1.9475833	total: 3.98s	remaining: 9.52s
295:	learn: 1.9475757	total: 4s	remaining: 9.51s
296:	learn: 1.9473430	total: 4.01s	remaining: 9.49s
297:	learn: 1.9465989	total: 4.02s	remaining: 9.48s
298:	learn: 1.9465693	total: 4.04s	remaining: 9.46s
299:	learn: 1.9465626	total: 4.05s	remaining: 9.45s
300:	learn: 1.9461329	total: 4.06s	remaining: 9.43s
301:	learn: 1.9460058	total: 4.08s	remaining: 9.42s
302:	learn: 1.9455290	total: 4.09s	remaining: 9.41s
303:	learn: 1.94

446:	learn: 1.9068124	total: 5.99s	remaining: 7.41s
447:	learn: 1.9066852	total: 6.01s	remaining: 7.4s
448:	learn: 1.9066834	total: 6.02s	remaining: 7.39s
449:	learn: 1.9066786	total: 6.04s	remaining: 7.38s
450:	learn: 1.9066774	total: 6.05s	remaining: 7.37s
451:	learn: 1.9066772	total: 6.07s	remaining: 7.36s
452:	learn: 1.9065988	total: 6.08s	remaining: 7.34s
453:	learn: 1.9063475	total: 6.09s	remaining: 7.33s
454:	learn: 1.9063003	total: 6.11s	remaining: 7.31s
455:	learn: 1.9062581	total: 6.12s	remaining: 7.3s
456:	learn: 1.9062581	total: 6.13s	remaining: 7.29s
457:	learn: 1.9062576	total: 6.14s	remaining: 7.27s
458:	learn: 1.9062576	total: 6.16s	remaining: 7.26s
459:	learn: 1.9062510	total: 6.17s	remaining: 7.24s
460:	learn: 1.9062328	total: 6.18s	remaining: 7.23s
461:	learn: 1.9062095	total: 6.2s	remaining: 7.22s
462:	learn: 1.9061865	total: 6.21s	remaining: 7.2s
463:	learn: 1.9060861	total: 6.22s	remaining: 7.19s
464:	learn: 1.9051283	total: 6.24s	remaining: 7.18s
465:	learn: 1.90

606:	learn: 1.8738345	total: 8.12s	remaining: 5.25s
607:	learn: 1.8735478	total: 8.13s	remaining: 5.24s
608:	learn: 1.8727283	total: 8.14s	remaining: 5.23s
609:	learn: 1.8714412	total: 8.16s	remaining: 5.21s
610:	learn: 1.8712268	total: 8.17s	remaining: 5.2s
611:	learn: 1.8712153	total: 8.18s	remaining: 5.19s
612:	learn: 1.8711810	total: 8.2s	remaining: 5.17s
613:	learn: 1.8709585	total: 8.21s	remaining: 5.16s
614:	learn: 1.8709562	total: 8.22s	remaining: 5.15s
615:	learn: 1.8706911	total: 8.23s	remaining: 5.13s
616:	learn: 1.8706814	total: 8.25s	remaining: 5.12s
617:	learn: 1.8706802	total: 8.26s	remaining: 5.11s
618:	learn: 1.8706801	total: 8.27s	remaining: 5.09s
619:	learn: 1.8706799	total: 8.29s	remaining: 5.08s
620:	learn: 1.8704262	total: 8.3s	remaining: 5.06s
621:	learn: 1.8698088	total: 8.31s	remaining: 5.05s
622:	learn: 1.8693715	total: 8.33s	remaining: 5.04s
623:	learn: 1.8692006	total: 8.35s	remaining: 5.03s
624:	learn: 1.8691893	total: 8.36s	remaining: 5.01s
625:	learn: 1.8

766:	learn: 1.8388901	total: 10.2s	remaining: 3.1s
767:	learn: 1.8388408	total: 10.2s	remaining: 3.09s
768:	learn: 1.8386401	total: 10.2s	remaining: 3.08s
769:	learn: 1.8381536	total: 10.3s	remaining: 3.06s
770:	learn: 1.8369474	total: 10.3s	remaining: 3.05s
771:	learn: 1.8368967	total: 10.3s	remaining: 3.04s
772:	learn: 1.8367474	total: 10.3s	remaining: 3.02s
773:	learn: 1.8367473	total: 10.3s	remaining: 3.01s
774:	learn: 1.8366974	total: 10.3s	remaining: 3s
775:	learn: 1.8365004	total: 10.3s	remaining: 2.98s
776:	learn: 1.8364328	total: 10.4s	remaining: 2.97s
777:	learn: 1.8364228	total: 10.4s	remaining: 2.96s
778:	learn: 1.8363456	total: 10.4s	remaining: 2.94s
779:	learn: 1.8363273	total: 10.4s	remaining: 2.93s
780:	learn: 1.8359578	total: 10.4s	remaining: 2.92s
781:	learn: 1.8355000	total: 10.4s	remaining: 2.9s
782:	learn: 1.8354955	total: 10.4s	remaining: 2.89s
783:	learn: 1.8354922	total: 10.4s	remaining: 2.88s
784:	learn: 1.8353442	total: 10.5s	remaining: 2.86s
785:	learn: 1.835

926:	learn: 1.8042706	total: 12.3s	remaining: 971ms
927:	learn: 1.8041152	total: 12.3s	remaining: 958ms
928:	learn: 1.8040961	total: 12.4s	remaining: 945ms
929:	learn: 1.8040958	total: 12.4s	remaining: 931ms
930:	learn: 1.8039140	total: 12.4s	remaining: 918ms
931:	learn: 1.8039058	total: 12.4s	remaining: 905ms
932:	learn: 1.8039024	total: 12.4s	remaining: 891ms
933:	learn: 1.8039014	total: 12.4s	remaining: 878ms
934:	learn: 1.8039013	total: 12.4s	remaining: 864ms
935:	learn: 1.8038103	total: 12.4s	remaining: 851ms
936:	learn: 1.8035783	total: 12.5s	remaining: 838ms
937:	learn: 1.8034998	total: 12.5s	remaining: 824ms
938:	learn: 1.8032956	total: 12.5s	remaining: 811ms
939:	learn: 1.8032488	total: 12.5s	remaining: 798ms
940:	learn: 1.8025157	total: 12.5s	remaining: 784ms
941:	learn: 1.8025154	total: 12.5s	remaining: 771ms
942:	learn: 1.8025010	total: 12.5s	remaining: 758ms
943:	learn: 1.8020710	total: 12.5s	remaining: 744ms
944:	learn: 1.8020176	total: 12.6s	remaining: 731ms
945:	learn: 

In [66]:
final_pred_1_2 = pred_to_df(final_pred_1, squared=True)

### Stacking 2

In [68]:
base_models_2 = [
    ('lgbm', lgbm),
    ('catboost', catboostm),
    ('xgboost', xgboost),
    ('randomforest', randomforest)
]

In [69]:
final_model_2, final_pred_2 = stacking_models(base_models_2, 
                                              X_train, y_train_transformed, 
                                              test=X_test, final_regressor='linear', 
                                              final_params={})

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-ch

138:	learn: 1.9891141	total: 2.4s	remaining: 14.9s
139:	learn: 1.9872972	total: 2.42s	remaining: 14.9s
140:	learn: 1.9856725	total: 2.44s	remaining: 14.9s
141:	learn: 1.9829191	total: 2.46s	remaining: 14.8s
142:	learn: 1.9779575	total: 2.47s	remaining: 14.8s
143:	learn: 1.9751523	total: 2.49s	remaining: 14.8s
144:	learn: 1.9706630	total: 2.5s	remaining: 14.7s
145:	learn: 1.9671126	total: 2.52s	remaining: 14.7s
146:	learn: 1.9652524	total: 2.53s	remaining: 14.7s
147:	learn: 1.9619327	total: 2.54s	remaining: 14.7s
148:	learn: 1.9601310	total: 2.56s	remaining: 14.6s
149:	learn: 1.9589545	total: 2.58s	remaining: 14.6s
150:	learn: 1.9551591	total: 2.59s	remaining: 14.6s
151:	learn: 1.9506114	total: 2.6s	remaining: 14.5s
152:	learn: 1.9483762	total: 2.62s	remaining: 14.5s
153:	learn: 1.9446391	total: 2.64s	remaining: 14.5s
154:	learn: 1.9417674	total: 2.65s	remaining: 14.5s
155:	learn: 1.9389325	total: 2.67s	remaining: 14.4s
156:	learn: 1.9388090	total: 2.69s	remaining: 14.4s
157:	learn: 1.9

306:	learn: 1.8136887	total: 4.97s	remaining: 11.2s
307:	learn: 1.8128219	total: 4.99s	remaining: 11.2s
308:	learn: 1.8113340	total: 5.01s	remaining: 11.2s
309:	learn: 1.8107824	total: 5.02s	remaining: 11.2s
310:	learn: 1.8100839	total: 5.04s	remaining: 11.2s
311:	learn: 1.8093794	total: 5.05s	remaining: 11.1s
312:	learn: 1.8084826	total: 5.07s	remaining: 11.1s
313:	learn: 1.8082373	total: 5.08s	remaining: 11.1s
314:	learn: 1.8077990	total: 5.09s	remaining: 11.1s
315:	learn: 1.8077766	total: 5.11s	remaining: 11.1s
316:	learn: 1.8069710	total: 5.13s	remaining: 11s
317:	learn: 1.8059846	total: 5.14s	remaining: 11s
318:	learn: 1.8049258	total: 5.16s	remaining: 11s
319:	learn: 1.8043324	total: 5.17s	remaining: 11s
320:	learn: 1.8034837	total: 5.19s	remaining: 11s
321:	learn: 1.8029842	total: 5.21s	remaining: 11s
322:	learn: 1.8024648	total: 5.22s	remaining: 10.9s
323:	learn: 1.8021273	total: 5.24s	remaining: 10.9s
324:	learn: 1.8021262	total: 5.25s	remaining: 10.9s
325:	learn: 1.8021194	to

473:	learn: 1.7553185	total: 7.51s	remaining: 8.34s
474:	learn: 1.7551719	total: 7.53s	remaining: 8.32s
475:	learn: 1.7549059	total: 7.54s	remaining: 8.3s
476:	learn: 1.7544021	total: 7.56s	remaining: 8.29s
477:	learn: 1.7543940	total: 7.57s	remaining: 8.27s
478:	learn: 1.7543767	total: 7.59s	remaining: 8.25s
479:	learn: 1.7543757	total: 7.6s	remaining: 8.24s
480:	learn: 1.7543752	total: 7.62s	remaining: 8.22s
481:	learn: 1.7543742	total: 7.63s	remaining: 8.2s
482:	learn: 1.7543295	total: 7.65s	remaining: 8.19s
483:	learn: 1.7543187	total: 7.66s	remaining: 8.17s
484:	learn: 1.7542598	total: 7.68s	remaining: 8.15s
485:	learn: 1.7533241	total: 7.7s	remaining: 8.14s
486:	learn: 1.7532856	total: 7.71s	remaining: 8.12s
487:	learn: 1.7532782	total: 7.73s	remaining: 8.11s
488:	learn: 1.7532523	total: 7.74s	remaining: 8.09s
489:	learn: 1.7531935	total: 7.76s	remaining: 8.07s
490:	learn: 1.7531780	total: 7.77s	remaining: 8.05s
491:	learn: 1.7531712	total: 7.79s	remaining: 8.04s
492:	learn: 1.75

644:	learn: 1.7356947	total: 10.1s	remaining: 5.55s
645:	learn: 1.7356944	total: 10.1s	remaining: 5.54s
646:	learn: 1.7356934	total: 10.1s	remaining: 5.52s
647:	learn: 1.7356769	total: 10.1s	remaining: 5.5s
648:	learn: 1.7353705	total: 10.1s	remaining: 5.49s
649:	learn: 1.7353702	total: 10.2s	remaining: 5.47s
650:	learn: 1.7353602	total: 10.2s	remaining: 5.46s
651:	learn: 1.7353596	total: 10.2s	remaining: 5.44s
652:	learn: 1.7353594	total: 10.2s	remaining: 5.42s
653:	learn: 1.7353591	total: 10.2s	remaining: 5.41s
654:	learn: 1.7353587	total: 10.2s	remaining: 5.39s
655:	learn: 1.7353587	total: 10.3s	remaining: 5.38s
656:	learn: 1.7353587	total: 10.3s	remaining: 5.36s
657:	learn: 1.7353587	total: 10.3s	remaining: 5.34s
658:	learn: 1.7353587	total: 10.3s	remaining: 5.33s
659:	learn: 1.7353586	total: 10.3s	remaining: 5.31s
660:	learn: 1.7353576	total: 10.3s	remaining: 5.3s
661:	learn: 1.7353576	total: 10.3s	remaining: 5.28s
662:	learn: 1.7353576	total: 10.4s	remaining: 5.27s
663:	learn: 1.

...

300:	learn: 1.9461329	total: 4.35s	remaining: 10.1s
301:	learn: 1.9460058	total: 4.36s	remaining: 10.1s
302:	learn: 1.9455290	total: 4.38s	remaining: 10.1s
303:	learn: 1.9454718	total: 4.39s	remaining: 10.1s
304:	learn: 1.9453515	total: 4.4s	remaining: 10s
305:	learn: 1.9449793	total: 4.42s	remaining: 10s
306:	learn: 1.9445849	total: 4.43s	remaining: 10s
307:	learn: 1.9437942	total: 4.45s	remaining: 9.99s
308:	learn: 1.9436138	total: 4.46s	remaining: 9.97s
309:	learn: 1.9425504	total: 4.47s	remaining: 9.96s
310:	learn: 1.9419266	total: 4.49s	remaining: 9.94s
311:	learn: 1.9413009	total: 4.5s	remaining: 9.92s
312:	learn: 1.9409381	total: 4.51s	remaining: 9.91s
313:	learn: 1.9399873	total: 4.53s	remaining: 9.89s
314:	learn: 1.9389907	total: 4.54s	remaining: 9.88s
315:	learn: 1.9379265	total: 4.56s	remaining: 9.86s
316:	learn: 1.9365412	total: 4.57s	remaining: 9.85s
317:	learn: 1.9359723	total: 4.59s	remaining: 9.84s
318:	learn: 1.9358662	total: 4.6s	remaining: 9.82s
319:	learn: 1.9348630

470:	learn: 1.9019736	total: 6.71s	remaining: 7.54s
471:	learn: 1.8999793	total: 6.73s	remaining: 7.53s
472:	learn: 1.8999765	total: 6.74s	remaining: 7.51s
473:	learn: 1.8999228	total: 6.75s	remaining: 7.5s
474:	learn: 1.8998732	total: 6.77s	remaining: 7.48s
475:	learn: 1.8998582	total: 6.78s	remaining: 7.47s
476:	learn: 1.8998482	total: 6.8s	remaining: 7.45s
477:	learn: 1.8998290	total: 6.81s	remaining: 7.44s
478:	learn: 1.8998257	total: 6.82s	remaining: 7.42s
479:	learn: 1.8997800	total: 6.83s	remaining: 7.41s
480:	learn: 1.8991513	total: 6.85s	remaining: 7.39s
481:	learn: 1.8991512	total: 6.86s	remaining: 7.38s
482:	learn: 1.8991511	total: 6.88s	remaining: 7.36s
483:	learn: 1.8988516	total: 6.89s	remaining: 7.35s
484:	learn: 1.8988391	total: 6.91s	remaining: 7.33s
485:	learn: 1.8978462	total: 6.92s	remaining: 7.32s
486:	learn: 1.8978389	total: 6.93s	remaining: 7.3s
487:	learn: 1.8978389	total: 6.95s	remaining: 7.29s
488:	learn: 1.8978375	total: 6.96s	remaining: 7.28s
489:	learn: 1.8

635:	learn: 1.8659769	total: 9.04s	remaining: 5.17s
636:	learn: 1.8655067	total: 9.05s	remaining: 5.16s
637:	learn: 1.8654249	total: 9.07s	remaining: 5.14s
638:	learn: 1.8641644	total: 9.08s	remaining: 5.13s
639:	learn: 1.8638726	total: 9.09s	remaining: 5.12s
640:	learn: 1.8634715	total: 9.11s	remaining: 5.1s
641:	learn: 1.8634380	total: 9.12s	remaining: 5.08s
642:	learn: 1.8632406	total: 9.13s	remaining: 5.07s
643:	learn: 1.8629372	total: 9.15s	remaining: 5.06s
644:	learn: 1.8627999	total: 9.16s	remaining: 5.04s
645:	learn: 1.8627666	total: 9.17s	remaining: 5.03s
646:	learn: 1.8624653	total: 9.19s	remaining: 5.01s
647:	learn: 1.8622668	total: 9.2s	remaining: 5s
648:	learn: 1.8622238	total: 9.21s	remaining: 4.98s
649:	learn: 1.8622237	total: 9.23s	remaining: 4.97s
650:	learn: 1.8621834	total: 9.24s	remaining: 4.95s
651:	learn: 1.8620997	total: 9.26s	remaining: 4.94s
652:	learn: 1.8620988	total: 9.27s	remaining: 4.92s
653:	learn: 1.8620982	total: 9.28s	remaining: 4.91s
654:	learn: 1.862

806:	learn: 1.8295541	total: 11.4s	remaining: 2.73s
807:	learn: 1.8295467	total: 11.4s	remaining: 2.71s
808:	learn: 1.8295358	total: 11.4s	remaining: 2.7s
809:	learn: 1.8293093	total: 11.4s	remaining: 2.68s
810:	learn: 1.8287489	total: 11.5s	remaining: 2.67s
811:	learn: 1.8287373	total: 11.5s	remaining: 2.66s
812:	learn: 1.8287371	total: 11.5s	remaining: 2.64s
813:	learn: 1.8287363	total: 11.5s	remaining: 2.63s
814:	learn: 1.8287346	total: 11.5s	remaining: 2.61s
815:	learn: 1.8287339	total: 11.5s	remaining: 2.6s
816:	learn: 1.8287339	total: 11.5s	remaining: 2.58s
817:	learn: 1.8287335	total: 11.6s	remaining: 2.57s
818:	learn: 1.8286987	total: 11.6s	remaining: 2.56s
819:	learn: 1.8281363	total: 11.6s	remaining: 2.54s
820:	learn: 1.8280479	total: 11.6s	remaining: 2.53s
821:	learn: 1.8276493	total: 11.6s	remaining: 2.51s
822:	learn: 1.8268274	total: 11.6s	remaining: 2.5s
823:	learn: 1.8264169	total: 11.6s	remaining: 2.48s
824:	learn: 1.8263886	total: 11.6s	remaining: 2.47s
825:	learn: 1.8

976:	learn: 1.7974092	total: 13.8s	remaining: 324ms
977:	learn: 1.7972362	total: 13.8s	remaining: 310ms
978:	learn: 1.7970489	total: 13.8s	remaining: 296ms
979:	learn: 1.7970210	total: 13.8s	remaining: 282ms
980:	learn: 1.7962745	total: 13.8s	remaining: 268ms
981:	learn: 1.7962735	total: 13.8s	remaining: 254ms
982:	learn: 1.7957767	total: 13.8s	remaining: 239ms
983:	learn: 1.7956106	total: 13.9s	remaining: 225ms
984:	learn: 1.7955305	total: 13.9s	remaining: 211ms
985:	learn: 1.7954301	total: 13.9s	remaining: 197ms
986:	learn: 1.7953321	total: 13.9s	remaining: 183ms
987:	learn: 1.7952687	total: 13.9s	remaining: 169ms
988:	learn: 1.7952119	total: 13.9s	remaining: 155ms
989:	learn: 1.7949192	total: 13.9s	remaining: 141ms
990:	learn: 1.7943850	total: 14s	remaining: 127ms
991:	learn: 1.7937503	total: 14s	remaining: 113ms
992:	learn: 1.7936679	total: 14s	remaining: 98.6ms
993:	learn: 1.7936667	total: 14s	remaining: 84.5ms
994:	learn: 1.7936611	total: 14s	remaining: 70.4ms
995:	learn: 1.79364

In [70]:
final_pred_2_2 = pred_to_df(final_pred_2, squared=True)

### Stacking 3

In [71]:
catboostm2, _ = prediction_models(X_train, y_train_transformed, test=X_test, model='catboost', params=dict_model_param['catboost_opt'])

0:	learn: 13.7584133	test: 13.8351663	best: 13.8351663 (0)	total: 12.4ms	remaining: 57.1s
100:	learn: 3.8602765	test: 3.8970616	best: 3.8970616 (100)	total: 1.19s	remaining: 52.8s
200:	learn: 3.5888459	test: 3.7620445	best: 3.7618049 (199)	total: 2.33s	remaining: 50.9s
300:	learn: 3.4011340	test: 3.6787959	best: 3.6787959 (300)	total: 3.47s	remaining: 49.6s
400:	learn: 3.2569407	test: 3.6197181	best: 3.6197181 (400)	total: 4.62s	remaining: 48.3s
500:	learn: 3.1378183	test: 3.5767599	best: 3.5763337 (499)	total: 5.77s	remaining: 47.2s
600:	learn: 3.0264306	test: 3.5320803	best: 3.5320803 (600)	total: 6.93s	remaining: 46.1s
700:	learn: 2.9348615	test: 3.5048298	best: 3.5048030 (699)	total: 8.08s	remaining: 44.9s
800:	learn: 2.8482367	test: 3.4802005	best: 3.4802005 (800)	total: 9.25s	remaining: 43.9s
900:	learn: 2.7720002	test: 3.4506593	best: 3.4506593 (900)	total: 10.4s	remaining: 42.7s
1000:	learn: 2.6994840	test: 3.4261103	best: 3.4261103 (1000)	total: 11.6s	remaining: 41.6s
1100:	le

In [72]:
xgboost2, _ = prediction_models(X_train, y_train_transformed, test=X_test, model='xgboost', params=dict_model_param['xgboost3'])

In [73]:
base_models_3 = [
    ('lgbm', lgbm),
    ('catboost', catboostm),
    ('catboost2', catboostm2),
    ('xgboost', xgboost2)
]

In [74]:
final_model_3, final_pred_3 = stacking_models(base_models_3,
                                                X_train, y_train_transformed,
                                                test=X_test, final_regressor='SVR', 
                                                final_params={})

[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Warning] feature_fraction is set=0.8244973703390805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8244973703390805
[LightGBM] [Warning] bagging_fraction is set=0.753170749901516, subsample=1.0 will be ignored. Current value: bagging_fraction=0.753170749901516
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-ch

140:	learn: 1.9856725	total: 2.63s	remaining: 16s
141:	learn: 1.9829191	total: 2.65s	remaining: 16s
142:	learn: 1.9779575	total: 2.66s	remaining: 16s
143:	learn: 1.9751523	total: 2.68s	remaining: 15.9s
144:	learn: 1.9706630	total: 2.7s	remaining: 15.9s
145:	learn: 1.9671126	total: 2.71s	remaining: 15.9s
146:	learn: 1.9652524	total: 2.73s	remaining: 15.8s
147:	learn: 1.9619327	total: 2.75s	remaining: 15.8s
148:	learn: 1.9601310	total: 2.76s	remaining: 15.8s
149:	learn: 1.9589545	total: 2.78s	remaining: 15.8s
150:	learn: 1.9551591	total: 2.8s	remaining: 15.7s
151:	learn: 1.9506114	total: 2.81s	remaining: 15.7s
152:	learn: 1.9483762	total: 2.83s	remaining: 15.7s
153:	learn: 1.9446391	total: 2.85s	remaining: 15.7s
154:	learn: 1.9417674	total: 2.87s	remaining: 15.6s
155:	learn: 1.9389325	total: 2.88s	remaining: 15.6s
156:	learn: 1.9388090	total: 2.9s	remaining: 15.6s
157:	learn: 1.9388017	total: 2.92s	remaining: 15.5s
158:	learn: 1.9373594	total: 2.93s	remaining: 15.5s
159:	learn: 1.9359200

303:	learn: 1.8149142	total: 5.45s	remaining: 12.5s
304:	learn: 1.8142967	total: 5.47s	remaining: 12.5s
305:	learn: 1.8142815	total: 5.48s	remaining: 12.4s
306:	learn: 1.8136887	total: 5.5s	remaining: 12.4s
307:	learn: 1.8128219	total: 5.52s	remaining: 12.4s
308:	learn: 1.8113340	total: 5.54s	remaining: 12.4s
309:	learn: 1.8107824	total: 5.55s	remaining: 12.4s
310:	learn: 1.8100839	total: 5.57s	remaining: 12.3s
311:	learn: 1.8093794	total: 5.59s	remaining: 12.3s
312:	learn: 1.8084826	total: 5.61s	remaining: 12.3s
313:	learn: 1.8082373	total: 5.62s	remaining: 12.3s
314:	learn: 1.8077990	total: 5.64s	remaining: 12.3s
315:	learn: 1.8077766	total: 5.66s	remaining: 12.3s
316:	learn: 1.8069710	total: 5.68s	remaining: 12.2s
317:	learn: 1.8059846	total: 5.69s	remaining: 12.2s
318:	learn: 1.8049258	total: 5.71s	remaining: 12.2s
319:	learn: 1.8043324	total: 5.73s	remaining: 12.2s
320:	learn: 1.8034837	total: 5.75s	remaining: 12.2s
321:	learn: 1.8029842	total: 5.77s	remaining: 12.1s
322:	learn: 1

472:	learn: 1.7557695	total: 8.41s	remaining: 9.38s
473:	learn: 1.7553185	total: 8.43s	remaining: 9.36s
474:	learn: 1.7551719	total: 8.45s	remaining: 9.34s
475:	learn: 1.7549059	total: 8.47s	remaining: 9.32s
476:	learn: 1.7544021	total: 8.49s	remaining: 9.31s
477:	learn: 1.7543940	total: 8.51s	remaining: 9.29s
478:	learn: 1.7543767	total: 8.52s	remaining: 9.27s
479:	learn: 1.7543757	total: 8.54s	remaining: 9.25s
480:	learn: 1.7543752	total: 8.56s	remaining: 9.23s
481:	learn: 1.7543742	total: 8.57s	remaining: 9.21s
482:	learn: 1.7543295	total: 8.59s	remaining: 9.2s
483:	learn: 1.7543187	total: 8.61s	remaining: 9.18s
484:	learn: 1.7542598	total: 8.63s	remaining: 9.16s
485:	learn: 1.7533241	total: 8.65s	remaining: 9.14s
486:	learn: 1.7532856	total: 8.66s	remaining: 9.13s
487:	learn: 1.7532782	total: 8.68s	remaining: 9.11s
488:	learn: 1.7532523	total: 8.7s	remaining: 9.09s
489:	learn: 1.7531935	total: 8.71s	remaining: 9.07s
490:	learn: 1.7531780	total: 8.73s	remaining: 9.05s
491:	learn: 1.

631:	learn: 1.7395069	total: 11.2s	remaining: 6.51s
632:	learn: 1.7395068	total: 11.2s	remaining: 6.49s
633:	learn: 1.7395058	total: 11.2s	remaining: 6.47s
634:	learn: 1.7394743	total: 11.2s	remaining: 6.46s
635:	learn: 1.7394593	total: 11.2s	remaining: 6.44s
636:	learn: 1.7393909	total: 11.3s	remaining: 6.42s
637:	learn: 1.7390701	total: 11.3s	remaining: 6.4s
638:	learn: 1.7381592	total: 11.3s	remaining: 6.38s
639:	learn: 1.7373696	total: 11.3s	remaining: 6.37s
640:	learn: 1.7373310	total: 11.3s	remaining: 6.35s
641:	learn: 1.7373293	total: 11.3s	remaining: 6.33s
642:	learn: 1.7369252	total: 11.4s	remaining: 6.31s
643:	learn: 1.7360217	total: 11.4s	remaining: 6.29s
644:	learn: 1.7356947	total: 11.4s	remaining: 6.28s
645:	learn: 1.7356944	total: 11.4s	remaining: 6.26s
646:	learn: 1.7356934	total: 11.4s	remaining: 6.24s
647:	learn: 1.7356769	total: 11.5s	remaining: 6.22s
648:	learn: 1.7353705	total: 11.5s	remaining: 6.2s
649:	learn: 1.7353702	total: 11.5s	remaining: 6.18s
650:	learn: 1.

...

3885:	learn: 1.6622577	total: 42.5s	remaining: 7.79s
3886:	learn: 1.6621162	total: 42.5s	remaining: 7.78s
3887:	learn: 1.6619575	total: 42.5s	remaining: 7.77s
3888:	learn: 1.6617091	total: 42.6s	remaining: 7.76s
3889:	learn: 1.6613773	total: 42.6s	remaining: 7.75s
3890:	learn: 1.6610000	total: 42.6s	remaining: 7.74s
3891:	learn: 1.6607826	total: 42.6s	remaining: 7.72s
3892:	learn: 1.6605309	total: 42.6s	remaining: 7.71s
3893:	learn: 1.6602491	total: 42.6s	remaining: 7.7s
3894:	learn: 1.6597298	total: 42.6s	remaining: 7.69s
3895:	learn: 1.6594609	total: 42.6s	remaining: 7.68s
3896:	learn: 1.6591807	total: 42.6s	remaining: 7.67s
3897:	learn: 1.6590017	total: 42.6s	remaining: 7.66s
3898:	learn: 1.6586533	total: 42.7s	remaining: 7.65s
3899:	learn: 1.6585056	total: 42.7s	remaining: 7.64s
3900:	learn: 1.6581753	total: 42.7s	remaining: 7.63s
3901:	learn: 1.6580271	total: 42.7s	remaining: 7.62s
3902:	learn: 1.6578896	total: 42.7s	remaining: 7.6s
3903:	learn: 1.6576197	total: 42.7s	remaining: 7

4058:	learn: 1.6229232	total: 44.4s	remaining: 5.9s
4059:	learn: 1.6226161	total: 44.4s	remaining: 5.88s
4060:	learn: 1.6224988	total: 44.4s	remaining: 5.87s
4061:	learn: 1.6223361	total: 44.4s	remaining: 5.86s
4062:	learn: 1.6222231	total: 44.4s	remaining: 5.85s
4063:	learn: 1.6219992	total: 44.5s	remaining: 5.84s
4064:	learn: 1.6216758	total: 44.5s	remaining: 5.83s
4065:	learn: 1.6213831	total: 44.5s	remaining: 5.82s
4066:	learn: 1.6212454	total: 44.5s	remaining: 5.81s
4067:	learn: 1.6209453	total: 44.5s	remaining: 5.8s
4068:	learn: 1.6207164	total: 44.5s	remaining: 5.79s
4069:	learn: 1.6205559	total: 44.5s	remaining: 5.78s
4070:	learn: 1.6203341	total: 44.5s	remaining: 5.76s
4071:	learn: 1.6199812	total: 44.5s	remaining: 5.75s
4072:	learn: 1.6197803	total: 44.6s	remaining: 5.74s
4073:	learn: 1.6196446	total: 44.6s	remaining: 5.73s
4074:	learn: 1.6194801	total: 44.6s	remaining: 5.72s
4075:	learn: 1.6191790	total: 44.6s	remaining: 5.71s
4076:	learn: 1.6190075	total: 44.6s	remaining: 5

4215:	learn: 1.5900889	total: 46.1s	remaining: 4.18s
4216:	learn: 1.5895451	total: 46.1s	remaining: 4.17s
4217:	learn: 1.5894611	total: 46.1s	remaining: 4.16s
4218:	learn: 1.5890647	total: 46.1s	remaining: 4.14s
4219:	learn: 1.5888550	total: 46.2s	remaining: 4.13s
4220:	learn: 1.5886112	total: 46.2s	remaining: 4.12s
4221:	learn: 1.5882828	total: 46.2s	remaining: 4.11s
4222:	learn: 1.5880170	total: 46.2s	remaining: 4.1s
4223:	learn: 1.5876345	total: 46.2s	remaining: 4.09s
4224:	learn: 1.5874193	total: 46.2s	remaining: 4.08s
4225:	learn: 1.5872490	total: 46.2s	remaining: 4.07s
4226:	learn: 1.5871343	total: 46.2s	remaining: 4.06s
4227:	learn: 1.5869324	total: 46.2s	remaining: 4.05s
4228:	learn: 1.5866579	total: 46.3s	remaining: 4.04s
4229:	learn: 1.5864473	total: 46.3s	remaining: 4.03s
4230:	learn: 1.5863385	total: 46.3s	remaining: 4.01s
4231:	learn: 1.5862178	total: 46.3s	remaining: 4s
4232:	learn: 1.5859497	total: 46.3s	remaining: 3.99s
4233:	learn: 1.5858296	total: 46.3s	remaining: 3.9

4382:	learn: 1.5535905	total: 48s	remaining: 2.35s
4383:	learn: 1.5533415	total: 48s	remaining: 2.34s
4384:	learn: 1.5531224	total: 48s	remaining: 2.33s
4385:	learn: 1.5529511	total: 48.1s	remaining: 2.32s
4386:	learn: 1.5526159	total: 48.1s	remaining: 2.31s
4387:	learn: 1.5523471	total: 48.1s	remaining: 2.3s
4388:	learn: 1.5519584	total: 48.1s	remaining: 2.29s
4389:	learn: 1.5518454	total: 48.1s	remaining: 2.28s
4390:	learn: 1.5517431	total: 48.1s	remaining: 2.27s
4391:	learn: 1.5516598	total: 48.1s	remaining: 2.26s
4392:	learn: 1.5515087	total: 48.1s	remaining: 2.25s
4393:	learn: 1.5513358	total: 48.1s	remaining: 2.23s
4394:	learn: 1.5511913	total: 48.1s	remaining: 2.22s
4395:	learn: 1.5508150	total: 48.2s	remaining: 2.21s
4396:	learn: 1.5505969	total: 48.2s	remaining: 2.2s
4397:	learn: 1.5504130	total: 48.2s	remaining: 2.19s
4398:	learn: 1.5502921	total: 48.2s	remaining: 2.18s
4399:	learn: 1.5500462	total: 48.2s	remaining: 2.17s
4400:	learn: 1.5499033	total: 48.2s	remaining: 2.16s
4

4551:	learn: 1.5207589	total: 49.9s	remaining: 504ms
4552:	learn: 1.5206487	total: 49.9s	remaining: 493ms
4553:	learn: 1.5203637	total: 49.9s	remaining: 482ms
4554:	learn: 1.5199987	total: 49.9s	remaining: 471ms
4555:	learn: 1.5198553	total: 49.9s	remaining: 460ms
4556:	learn: 1.5196810	total: 49.9s	remaining: 449ms
4557:	learn: 1.5194596	total: 50s	remaining: 438ms
4558:	learn: 1.5191488	total: 50s	remaining: 427ms
4559:	learn: 1.5190243	total: 50s	remaining: 416ms
4560:	learn: 1.5188639	total: 50s	remaining: 405ms
4561:	learn: 1.5187277	total: 50s	remaining: 395ms
4562:	learn: 1.5182968	total: 50s	remaining: 384ms
4563:	learn: 1.5181487	total: 50s	remaining: 373ms
4564:	learn: 1.5179707	total: 50s	remaining: 362ms
4565:	learn: 1.5178148	total: 50s	remaining: 351ms
4566:	learn: 1.5176832	total: 50.1s	remaining: 340ms
4567:	learn: 1.5175402	total: 50.1s	remaining: 329ms
4568:	learn: 1.5173323	total: 50.1s	remaining: 318ms
4569:	learn: 1.5171993	total: 50.1s	remaining: 307ms
4570:	learn

In [75]:
final_pred_3_2 = pred_to_df(final_pred_3, True)

## Postprocessing

In [76]:
def merge_with_slided(dataframe, merging_dataframe):
    
    new_dataframe = pd.DataFrame()
    new_dataframe['val1'] = dataframe['values']
    
    new_dataframe['val2'] = merging_dataframe['values']
    new_dataframe['max'] = new_dataframe[['val1','val2']].max(axis=1)
    new_dataframe['values'] = np.where(new_dataframe['val1'] < 3000, new_dataframe['val1'], new_dataframe['max'])
    only_A=new_dataframe.copy()[:720]
    only_B_C=new_dataframe.copy()[720:]
    only_B_C['values'] = np.where(only_B_C['val1'] < 550, only_B_C['val1'], only_B_C['max'])
    new_dataframe = pd.concat([only_A, only_B_C])['values']
    
    return pred_to_df(new_dataframe)

In [77]:
def merge_with_slided_A(dataframe, merging_dataframe):
    
    new_dataframe = pd.DataFrame()
    new_dataframe['val1'] = dataframe['values']
    
    new_dataframe['val2'] = merging_dataframe['values']
    new_dataframe['max'] = new_dataframe[['val1','val2']].max(axis=1)
    new_dataframe['values'] = np.where(new_dataframe['val1'] < 3000, new_dataframe['val1'], new_dataframe['max'])
    only_A=new_dataframe.copy()[:720]
    only_B_C=new_dataframe.copy()[720:]
    new_dataframe = pd.concat([only_A, only_B_C])['values']
    
    return pred_to_df(new_dataframe)

In [78]:
merge1 = merge_with_slided_A(result_lgbm, result_lgbm_slided)

In [79]:
merge2 = merge_with_slided_A(merge1, result_lgbm_r_slided)

In [80]:
merge3 = merge_with_slided(final_pred_1_2, merge2)

In [81]:
merge3_1 = merge_with_slided(final_pred_2_2, merge2)

In [82]:
#AVG of two models
result_column = (merge3['values'] + merge3_1['values']) / 2

avg1 = pd.DataFrame({'Mean_Value': result_column})
avg1 = pred_to_df(avg1)

In [83]:
avg2 = pd.DataFrame()
avg2['val1'] = avg1['values'] 
avg2['val2'] = final_pred_3_2['values']
avg2['avg'] = (avg2['val1'] + avg2['val2'] * 2)/3
avg2['values'] = np.where(avg2['val1'] > 2000, avg2['val1'], avg2['avg'])
avg2 = avg2.drop(columns=['val1','val2','avg'])
avg2= pred_to_df(avg2)
avg2.to_csv('short_notebook_1_submission.csv')